# Pipeline et modèle

In [2]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

import pickle

In [3]:
df = pd.read_csv("../data/AmesHousing.csv")

Il ne faut selectionner que 10 features, on supprime bsmt Qual et garage Finish qui sont trop proches d'autres variables.<br>
Neighboorhood possède une valeur isolée, que l'on va enlever dans un premier temps pour pouvoir stratifier lors du train test split. <br>
On rajoutera après la ligne manquante à notre ensemble d'entraînement

In [4]:
df["Neighborhood"].value_counts()

NAmes      443
CollgCr    267
OldTown    239
Edwards    194
Somerst    182
NridgHt    166
Gilbert    165
Sawyer     151
NWAmes     131
SawyerW    125
Mitchel    114
BrkSide    108
Crawfor    103
IDOTRR      93
Timber      72
NoRidge     71
StoneBr     51
SWISU       48
ClearCr     44
MeadowV     37
BrDale      30
Blmngtn     28
Veenker     24
NPkVill     23
Blueste     10
Greens       8
GrnHill      2
Landmrk      1
Name: Neighborhood, dtype: int64

In [5]:
mask = df["Neighborhood"] == "Landmrk"

df_ngbd_unique = df[mask]
df_ngbd_unique

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
2788,2789,907230240,160,RH,NaN,3612,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,6,2006,WD,Normal,137000


In [6]:
numeric_features = ["Year Built", "Total Bsmt SF", "1st Flr SF", "Gr Liv Area", "Garage Area", "Overall Qual", "Full Bath"]
ordinal_features = [ "Exter Qual",  "Kitchen Qual"]
cat_feature = ["Neighborhood"]
all_col = numeric_features.copy()
all_col.extend(ordinal_features)
all_col.extend(cat_feature)

mask = df["Neighborhood"] != "Landmrk"
df=df[mask]

X = df[all_col]
y = np.log(df[["SalePrice"]])

X["Total Bsmt SF"].fillna(1052, inplace=True)
X["Garage Area"].fillna(472, inplace=True)

/tmp/ipykernel_41686/116759663.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["Total Bsmt SF"].fillna(1052, inplace=True)
/tmp/ipykernel_41686/116759663.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["Garage Area"].fillna(472, inplace=True)


In [7]:
X_train_add = pd.DataFrame(df_ngbd_unique[all_col])

# on oublie d'appliquer le logarithme
y_train_add = pd.DataFrame(np.log(df_ngbd_unique['SalePrice']))

In [8]:
y_train_add.columns

Index(['SalePrice'], dtype='object')

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=X["Neighborhood"] ,random_state=42)

X_train = X_train.append(X_train_add, ignore_index=True)
y_train = y_train.append(y_train_add, ignore_index=True)


/tmp/ipykernel_41686/3402445434.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_train = X_train.append(X_train_add, ignore_index=True)
/tmp/ipykernel_41686/3402445434.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_train = y_train.append(y_train_add, ignore_index=True)


In [13]:
numeric_transformer = SimpleImputer()

exter_cat = [ 'Po', 'Fa','TA', 'Gd','Ex']
kitchen_cat = [ 'Po', 'Fa','TA', 'Gd',"Ex"]

ordinal_transformer = OrdinalEncoder(categories=[exter_cat, kitchen_cat])

categorical_transformer = OneHotEncoder()

In [14]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('ord', ordinal_transformer, ordinal_features),
        ('cat', categorical_transformer, cat_feature)
    ]
)

In [15]:
reg = LinearRegression()

In [16]:
pipe = Pipeline([
     ('preprocessor', preprocessor),
     ('reg', reg)
])

pipe.fit(X_train, y_train) 

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['Year Built',
                                                   'Total Bsmt SF',
                                                   '1st Flr SF', 'Gr Liv Area',
                                                   'Garage Area',
                                                   'Overall Qual',
                                                   'Full Bath']),
                                                 ('ord',
                                                  OrdinalEncoder(categories=[['Po',
                                                                              'Fa',
                                                                              'TA',
                                                                              'Gd',
                                                                              'Ex'],
                                                                             ['Po',
                                                                              'Fa',
                                                                              'TA',
                                                                              'Gd',
                                                                              'Ex']]),
                                                  ['Exter Qual',
                                                   'Kitchen Qual']),
                                                 ('cat', OneHotEncoder(),
                                                  ['Neighborhood'])])),
                ('reg', LinearRegression())])

In [17]:
score = pipe.score(X_test,y_test)

predict_train = pipe.predict(X_train)
predict_test = pipe.predict(X_test)

mae_train = mean_absolute_error(np.exp(y_train), np.exp(predict_train))
mae_test = mean_absolute_error(np.exp(y_test), np.exp(predict_test))

# Root Mean Squared Error on train and test date
print('R2: ', score)
print('MAE on train data: ', mae_train)
print('MAE on test data: ', mae_test)

R2:  0.8617682068708926
MAE on train data:  18510.174994462286
MAE on test data:  18230.967168584102


In [18]:
# modele_df = pd.DataFrame(columns=['model', 'R2', 'MAE_train', 'MAE_test'])

# model_init = pd.read_csv("model-csv/LinearRegression initial.csv")

# modele_df = modele_df.append(model_init, ignore_index=True)

/tmp/ipykernel_41686/268407385.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  modele_df = modele_df.append(model_init, ignore_index=True)


In [19]:
# col_dict = {
#     'model': ["LinearRegression"],
#     'R2': [score],
#     'MAE_train': [mae_train],
#     'MAE_test': [mae_test]
# }

# new_model = pd.DataFrame(col_dict)

# name_model = new_model["model"].iloc[0]

# modele_df = modele_df.append(new_model, ignore_index=True)

# modele_df.to_csv(f"model-csv/{name_model}.csv", index=False)

/tmp/ipykernel_41686/2998562700.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  modele_df = modele_df.append(new_model, ignore_index=True)


In [20]:
# modele_df

,model,R2,MAE_train,MAE_test
0,LinearRegression initial,0.846420,20289.336397,21324.920124
1,LinearRegression,0.861768,18510.174994,18230.967169


In [21]:
# filename = '../real_estate_app/main_app/static/models/finalized_model.pkl'
# # pickle.dump(pipe, open(filename, 'wb'))